# Portofolio Day 1 - Compare some Class Assets

Hello there! My name is Febryan here is my linkedin (https://www.linkedin.com/in/febryanmughni/)

Goals: The aim of this project is to compare several asset classes such as BTC, GOLD, and SPX in the last 3 years

Enjoy!

In [4]:
# Imports

import yfinance as yf
import os
from dotenv import load_dotenv
from scraper import fetch_website_contents
from IPython.display import Markdown, display
from openai import OpenAI




In [5]:
btc = yf.download("BTC-USD", period="3y")
spx = yf.download("^GSPC", period="3y")
gold = yf.download("GC=F", period="3y")

print(btc)
print(spx)
print(gold)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Price              Close          High           Low          Open  \
Ticker           BTC-USD       BTC-USD       BTC-USD       BTC-USD   
Date                                                                 
2023-01-18  20688.781250  21564.501953  20541.544922  21161.050781   
2023-01-19  21086.792969  21163.011719  20685.380859  20686.746094   
2023-01-20  22676.552734  22692.357422  20919.126953  21085.373047   
2023-01-21  22777.625000  23282.347656  22511.833984  22677.427734   
2023-01-22  22720.416016  23056.730469  22387.900391  22777.986328   
...                  ...           ...           ...           ...   
2026-01-13  95321.781250  96011.625000  90941.929688  91185.335938   
2026-01-14  96929.328125  97860.601562  94583.046875  95322.906250   
2026-01-15  95551.187500  97150.171875  95103.242188  96931.289062   
2026-01-16  95525.117188  95801.890625  94259.273438  95554.101562   
2026-01-18  95019.250000  95186.789062  94854.273438  95112.398438   

Price            Vo

In [6]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


## Step-by-Step: Building an LLM Financial Analysis Agent

This guide will walk you through creating an LLM-powered financial analyst that can analyze and compare asset classes using your downloaded data.

### Step 1: Analyze and Prepare Financial Data
Extract key statistics (returns, volatility, price ranges) from BTC, Gold, and SPX data

### Step 2: Create Data Summary for LLM
Format the statistics into a text string that the LLM can easily process

### Step 3: Setup OpenAI Client
Initialize the OpenAI API client with your API key

### Step 4: Create System Prompt & User Prompt
Define the LLM's role and the specific analysis request

### Step 5: Call LLM API
Send the prompts to OpenAI and receive the analysis

### Step 6: Display Results
Show the LLM's analysis in a readable format

In [8]:
# Step 1: Analyze Financial Data
# Calculate key statistics from BTC, Gold, and SPX data

import pandas as pd

def analyze_asset(data, name):
    """Calculate key statistics for each asset"""
    # Handle MultiIndex columns from yfinance
    if isinstance(data.columns, pd.MultiIndex):
        # If MultiIndex, extract Close column for the specific ticker
        close_prices = data[('Close', name)]
    else:
        # If not MultiIndex, use Close column directly
        close_prices = data['Close']
    
    start_price = float(close_prices.iloc[0])
    end_price = float(close_prices.iloc[-1])
    total_return = ((end_price - start_price) / start_price) * 100
    max_price = float(close_prices.max())
    min_price = float(close_prices.min())
    volatility = float(close_prices.std())
    
    return {
        'Asset': name,
        'Start Price': start_price,
        'End Price': end_price,
        'Total Return (%)': round(total_return, 2),
        'Max Price': max_price,
        'Min Price': min_price,
        'Volatility': round(volatility, 2)
    }

# Analyze each asset
btc_stats = analyze_asset(btc, 'BTC-USD')
spx_stats = analyze_asset(spx, '^GSPC')
gold_stats = analyze_asset(gold, 'GC=F')

print("📊 Asset Statistics (3 Years):")
print("\n" + "="*60)
for stats in [btc_stats, spx_stats, gold_stats]:
    print(f"\n{stats['Asset']}:")
    print(f"  Return: {stats['Total Return (%)']}%")
    print(f"  Start Price: ${stats['Start Price']:,.2f}")
    print(f"  End Price: ${stats['End Price']:,.2f}")
    print(f"  Volatility: ${stats['Volatility']:,.2f}")

📊 Asset Statistics (3 Years):


BTC-USD:
  Return: 359.28%
  Start Price: $20,688.78
  End Price: $95,019.25
  Volatility: $31,428.03

^GSPC:
  Return: 73.89%
  Start Price: $3,990.97
  End Price: $6,940.01
  Volatility: $879.78

GC=F:
  Return: 140.58%
  Start Price: $1,907.20
  End Price: $4,588.40
  Volatility: $734.06


In [9]:
# Step 2: Create Data Summary for LLM
# Format data into a string that the LLM can easily understand

data_summary = f"""
3-YEAR FINANCIAL DATA (2023-2026):

BITCOIN (BTC-USD):
- Start Price: ${btc_stats['Start Price']:,.2f}
- End Price: ${btc_stats['End Price']:,.2f}
- Total Return: {btc_stats['Total Return (%)']}%
- Highest Price: ${btc_stats['Max Price']:,.2f}
- Lowest Price: ${btc_stats['Min Price']:,.2f}
- Volatility: ${btc_stats['Volatility']:,.2f}

S&P 500 (^GSPC):
- Start Price: ${spx_stats['Start Price']:,.2f}
- End Price: ${spx_stats['End Price']:,.2f}
- Total Return: {spx_stats['Total Return (%)']}%
- Highest Price: ${spx_stats['Max Price']:,.2f}
- Lowest Price: ${spx_stats['Min Price']:,.2f}
- Volatility: ${spx_stats['Volatility']:,.2f}

GOLD (GC=F):
- Start Price: ${gold_stats['Start Price']:,.2f}
- End Price: ${gold_stats['End Price']:,.2f}
- Total Return: {gold_stats['Total Return (%)']}%
- Highest Price: ${gold_stats['Max Price']:,.2f}
- Lowest Price: ${gold_stats['Min Price']:,.2f}
- Volatility: ${gold_stats['Volatility']:,.2f}
"""

print("📋 Data Summary to be sent to LLM:")
print(data_summary)

📋 Data Summary to be sent to LLM:

3-YEAR FINANCIAL DATA (2023-2026):

BITCOIN (BTC-USD):
- Start Price: $20,688.78
- End Price: $95,019.25
- Total Return: 359.28%
- Highest Price: $124,752.53
- Lowest Price: $20,187.24
- Volatility: $31,428.03

S&P 500 (^GSPC):
- Start Price: $3,990.97
- End Price: $6,940.01
- Total Return: 73.89%
- Highest Price: $6,977.27
- Lowest Price: $3,855.76
- Volatility: $879.78

GOLD (GC=F):
- Start Price: $1,907.20
- End Price: $4,588.40
- Total Return: 140.58%
- Highest Price: $4,626.30
- Lowest Price: $1,808.80
- Volatility: $734.06



In [10]:
# Step 3: Setup OpenAI Client
client = OpenAI(api_key=api_key)

print("✅ OpenAI Client is ready to use!")

✅ OpenAI Client is ready to use!


In [11]:
# Step 4: Create System Prompt & User Prompt

system_prompt = """You are a professional financial analyst specializing in asset class comparison. 
Your role is to analyze financial data and provide:
1. Clear comparison between different asset classes
2. Insights about performance, risk, and volatility
3. Recommendations based on the data
4. Context about market conditions

Use Markdown formatting for better readability with headers, bullet points, and clear sections."""

user_prompt = f"""Please analyze the following financial data and provide a comprehensive comparison between Bitcoin, S&P 500, and Gold:

{data_summary}

Please provide:
1. Performance comparison between the three assets
2. Risk and volatility analysis
3. Investment recommendations based on this data
4. Insights about market trends over the past 3 years

Format your response with clear sections and use Markdown formatting."""

print("✅ Prompts have been created!")

✅ Prompts have been created!


In [12]:
# Step 5: Call LLM API

# Create messages list for the API
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

print("🔄 Sending request to OpenAI...")

try:
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # Faster and more economical model
        messages=messages,
        temperature=0.7  # Balance between creativity and accuracy
    )
    
    print("✅ Response received successfully!")
    analysis_result = response.choices[0].message.content
    
except Exception as e:
    print(f"❌ Error: {e}")
    analysis_result = None

🔄 Sending request to OpenAI...
✅ Response received successfully!


In [13]:
# Step 6: Display Results

if analysis_result:
    print("\n" + "="*60)
    print("🤖 LLM ANALYSIS RESULTS")
    print("="*60 + "\n")
    display(Markdown(analysis_result))
else:
    print("No results to display.")


🤖 LLM ANALYSIS RESULTS



# Comprehensive Asset Class Comparison: Bitcoin, S&P 500, and Gold

## 1. Performance Comparison

### Total Returns
- **Bitcoin (BTC-USD)**
  - **Total Return:** 359.28%
  - **Price Movement:** From $20,688.78 to $95,019.25

- **S&P 500 (^GSPC)**
  - **Total Return:** 73.89%
  - **Price Movement:** From $3,990.97 to $6,940.01

- **Gold (GC=F)**
  - **Total Return:** 140.58%
  - **Price Movement:** From $1,907.20 to $4,588.40

### Summary of Performance
- Bitcoin has significantly outperformed both the S&P 500 and Gold, achieving a return of over 350% in just three years.
- Gold has shown respectable growth, more than doubling its value, while the S&P 500 has increased by nearly 74%.

## 2. Risk and Volatility Analysis

### Volatility
- **Bitcoin (BTC-USD)**
  - **Volatility:** $31,428.03 
  - **Implication:** Extremely high volatility; indicative of the speculative nature of crypto assets.

- **S&P 500 (^GSPC)**
  - **Volatility:** $879.78
  - **Implication:** Moderate volatility; reflects a diversified portfolio of large-cap stocks.

- **Gold (GC=F)**
  - **Volatility:** $734.06
  - **Implication:** Lower volatility compared to Bitcoin and slightly lower than the S&P 500; typically seen as a safe-haven asset.

### Risk Assessment
- **Bitcoin** presents the highest risk due to its extreme price fluctuations, making it suitable for high-risk investors looking for potentially high returns.
- **S&P 500** offers a balanced approach with moderate risk, suitable for long-term investors seeking steady growth.
- **Gold** is the least volatile and is generally favored by conservative investors looking to hedge against market uncertainty.

## 3. Investment Recommendations

### For High-Risk Tolerance
- **Invest in Bitcoin:** Given its astronomical growth, Bitcoin is appealing for those who can handle high volatility and are looking for substantial returns. 

### For Moderate Risk Tolerance
- **Invest in S&P 500:** The S&P 500 is ideal for investors looking for moderate growth with reasonable risk. It provides exposure to the broader economy and benefits from corporate earnings growth.

### For Low-Risk Tolerance
- **Invest in Gold:** Gold serves as a protective asset and is a reliable hedge against inflation and market downturns. This is suitable for conservative investors or those nearing retirement.

## 4. Insights About Market Trends Over the Past 3 Years

- **Bitcoin's Surge:** The significant increase in Bitcoin’s price reflects growing institutional adoption, a shift towards digital assets, and the increasing use of blockchain technology.

- **S&P 500 Performance:** The S&P 500's growth has been driven by a recovery from the COVID-19 pandemic, strong corporate earnings, and low-interest rates, despite facing challenges like inflation and geopolitical tensions.

- **Gold as a Safe Haven:** Gold's performance indicates its traditional role as a hedge against inflation and market uncertainty, especially during periods of high volatility in both the stock market and emerging asset classes like cryptocurrencies.

### Conclusion
The analysis showcases that each asset class has its own unique characteristics and risk profiles. Investors should align their investment choices with their risk tolerance, investment horizon, and market outlook. Diversification across these asset classes can also be a prudent strategy to balance risk and return.